# Classification Exercise

In [1]:
import pandas as pd

In [2]:
census = pd.read_csv('../census_data.csv')

In [3]:
census.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [4]:
census['income_bracket'].unique()

array([' <=50K', ' >50K'], dtype=object)

In [5]:
def label_fix(label):
    if label == ' <=50K':
        return 0
    else:
        return 1

In [6]:
census['income_bracket'] = census['income_bracket'].apply(label_fix)

In [7]:
census.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0


# Perform a Train Test Split on the Data

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
y_labels = census['income_bracket']
x_data = census.drop('income_bracket',axis=1)

In [10]:
X_train,X_test,y_train,y_test = train_test_split(x_data,y_labels,test_size=0.3,random_state=101)

# Create the Feature Columns for tf.estimator

In [11]:
census.columns

Index(['age', 'workclass', 'education', 'education_num', 'marital_status',
       'occupation', 'relationship', 'race', 'gender', 'capital_gain',
       'capital_loss', 'hours_per_week', 'native_country', 'income_bracket'],
      dtype='object')

In [12]:
import tensorflow as tf

In [13]:
# Create the tf.feature_columns for the categorical values. Use Vocabulary lists or just use hash buckets

In [14]:
gender = tf.feature_column.categorical_column_with_vocabulary_list('gender',['Female','Male'])

In [15]:
occupation = tf.feature_column.categorical_column_with_hash_bucket('occupation',hash_bucket_size=1000)

In [16]:
marital_status = tf.feature_column.categorical_column_with_hash_bucket('marital_status',hash_bucket_size=1000)
relationship = tf.feature_column.categorical_column_with_hash_bucket('relationship',hash_bucket_size=1000)
education = tf.feature_column.categorical_column_with_hash_bucket('education',hash_bucket_size=1000)
workclass = tf.feature_column.categorical_column_with_hash_bucket('workclass',hash_bucket_size=1000)
native_country = tf.feature_column.categorical_column_with_hash_bucket('native_country',hash_bucket_size=1000)

In [17]:
# Create the continuous feature_columns for the continuous values using numeric_column

In [18]:
age = tf.feature_column.numeric_column('age')

In [19]:
education_num = tf.feature_column.numeric_column('education_num')
capital_gain = tf.feature_column.numeric_column('capital_gain')
capital_loss = tf.feature_column.numeric_column('capital_loss')
hours_per_week = tf.feature_column.numeric_column('hours_per_week')

In [20]:
feat_cols = [gender,occupation,marital_status,relationship,education,workclass,native_country,
            age,education_num,capital_gain,capital_loss,hours_per_week]

In [21]:
# Create Input function
input_func = tf.estimator.inputs.pandas_input_fn(x = X_train,y = y_train,batch_size=100,num_epochs=None,shuffle=True)

# Create your model with tf.estimator

In [22]:
model = tf.estimator.LinearClassifier(feature_columns=feat_cols,n_classes=2)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\VINITJ~1\\AppData\\Local\\Temp\\tmp5pwwbqjl', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000167EEC27BA8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [23]:
model.train(input_fn=input_func,steps=1000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\VINITJ~1\AppData\Local\Temp\tmp5pwwbqjl\model.ckpt.
INFO:tensorflow:loss = 69.3147, step = 1
INFO:tensorflow:global_step/sec: 49.5792
INFO:tensorflow:loss = 1047.57, step = 101 (2.027 sec)
INFO:tensorflow:global_step/sec: 99.0109
INFO:tensorflow:loss = 682.758, step = 201 (1.009 sec)
INFO:tensorflow:global_step/sec: 94.6978
INFO:tensorflow:loss = 207.784, step = 301 (1.059 sec)
INFO:tensorflow:global_step/sec: 95.8786
INFO:tensorflow:loss = 171.074, step = 401 (1.041 sec)
INFO:tensorflow:global_step/sec: 84.8183
INFO:tensorflow:loss = 109.185, step = 501 (1.187 sec)
INFO:tensorflow:global_step/sec: 75.189
INFO:tensorflow:loss = 529.868, step = 601 (1.322 sec)
INFO:tensorflow:global_step/sec: 84.9625

# Predict your model

In [26]:
pred_fn = tf.estimator.inputs.pandas_input_fn(x = X_test,batch_size=len(X_test),shuffle=False)

In [27]:
pred_gen = model.predict(input_fn=pred_fn)

In [28]:
predictions = list(pred_gen)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\VINITJ~1\AppData\Local\Temp\tmp5pwwbqjl\model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [29]:
predictions

[{'class_ids': array([0], dtype=int64),
  'classes': array([b'0'], dtype=object),
  'logistic': array([ 0.22511026], dtype=float32),
  'logits': array([-1.23613048], dtype=float32),
  'probabilities': array([ 0.77488977,  0.22511026], dtype=float32)},
 {'class_ids': array([0], dtype=int64),
  'classes': array([b'0'], dtype=object),
  'logistic': array([ 0.0001902], dtype=float32),
  'logits': array([-8.56723595], dtype=float32),
  'probabilities': array([  9.99809802e-01,   1.90201565e-04], dtype=float32)},
 {'class_ids': array([1], dtype=int64),
  'classes': array([b'1'], dtype=object),
  'logistic': array([ 0.51616359], dtype=float32),
  'logits': array([ 0.064677], dtype=float32),
  'probabilities': array([ 0.48383635,  0.51616359], dtype=float32)},
 {'class_ids': array([0], dtype=int64),
  'classes': array([b'0'], dtype=object),
  'logistic': array([  3.59752594e-05], dtype=float32),
  'logits': array([-10.23264313], dtype=float32),
  'probabilities': array([  9.99963999e-01,   3.5

In [30]:
final_preds = [pred['class_ids'][0] for pred in predictions]

In [35]:
from sklearn.metrics import classification_report

In [34]:
classificaton_report(y_test,final_preds)

NameError: name 'classificaton_report' is not defined